In [1]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
#from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI


In [2]:
openai_api_key='sk-q6lmVPWCubXsYBU8'

In [5]:
llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
     {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="distilbert-base-nli-mean-tokens")

In [8]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, 
    #OpenAIEmbeddings(openai_api_key=openai_api_key), 
    embeddings,
    # This is the VectorStore  to store the embeddings and do a similarity search
    FAISS, 
    #  number of examples
    k=2
)

In [9]:
similar_prompt = FewShotPromptTemplate(
    # select examples
    example_selector=example_selector,
    # prompt
    example_prompt=example_prompt,
    # prompt alterations
    prefix="Give the location an item is usually found in",
    suffix="Input: {noun}\nOutput:",
    # variables of prompt
    input_variables=["noun"],
)

In [10]:
my_noun = "student"

print(similar_prompt.format(noun=my_noun))

Give the location an item is usually found in

Example Input: tree
Example Output: ground

Example Input: bird
Example Output: nest

Input: student
Output:


In [11]:
similar_prompt.format(noun=my_noun)

'Give the location an item is usually found in\n\nExample Input: tree\nExample Output: ground\n\nExample Input: bird\nExample Output: nest\n\nInput: student\nOutput:'

In [12]:
llm(similar_prompt.format(noun=my_noun))


' classroom'